base class for Gates 

In [1]:
# Define the base class for Gates 
class Gate: 
    def forward(self): 
        raise NotImplementedError 
    def backward(self): 
        raise NotImplementedError  

AddGate

In [2]:
# Example of an AddGate class inheriting from the Gate class 
import numpy as np
class AddGate(Gate): 
    def forward(self, x, y): 
        self.x = x 
        self.y = y 
        return x + y 
    def backward(self, dz): 
        dx = dz * np.ones_like(self.x) 
        dy = dz * np.ones_like(self.y) 
        return dx, dy 

 MultiplyGate

In [ ]:
# Example of a MultiplyGate class inheriting from the Gate class
class MultiplyGate(Gate):
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, dz):
        dx = dz * self.y
        dy = dz * self.x
        return dx, dy

Linear activation function

In [ ]:

# Example of a Linear activation function
class LinearActivation(Gate):
    def forward(self, x):
        self.x = x
        return x

    def backward(self, dz):
        dx = dz
        return dx

ReLU activation function

In [ ]:
# Example of a ReLU activation function
class ReLUActivation(Gate):
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, dz):
        dx = dz * np.where(self.x > 0, 1, 0)
        return dx

Sigmoid activation function

In [ ]:
# Example of a Sigmoid activation function
class SigmoidActivation(Gate):
    def forward(self, x):
        self.x = x
        return 1 / (1 + np.exp(-x))

    def backward(self, dz):
        sigmoid_x = 1 / (1 + np.exp(-self.x))
        dx = dz * sigmoid_x * (1 - sigmoid_x)
        return dx

Softmax activation function

In [ ]:
# Example of a Softmax activation function
class SoftmaxActivation(Gate):
    def forward(self, x):
        self.x = x
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def backward(self, dz):
        softmax_x = self.forward(self.x)
        dx = dz * softmax_x * (1 - softmax_x)
        return dx

Tanh activation function

In [ ]:
# Example of a Tanh activation function
class TanhActivation(Gate):
    def forward(self, x):
        self.x = x
        return np.tanh(x)

    def backward(self, dz):
        tanh_x = np.tanh(self.x)
        dx = dz * (1 - tanh_x ** 2)
        return dx

 Binary Cross-Entropy (BCE) loss function

In [ ]:
# Example of Binary Cross-Entropy (BCE) loss function
class BinaryCrossEntropyLoss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self):
        dx = (self.y_pred - self.y_true) / (self.y_pred * (1 - self.y_pred))
        return dx

L2 loss function

In [ ]:
# Example of L2 loss function
class L2Loss(Gate):
    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return 0.5 * np.mean((y_pred - y_true) ** 2)

    def backward(self):
        dx = self.y_pred - self.y_true
        return dx